# 信息提取
因为优秀的上下文理解能力，Base模型可以轻松的从大段数据中提取重要信息，在提示词的约束下得到对应的内容。

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.17it/s]


In [2]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

In [3]:
text="EcoFusion Pro是一款便携式搅拌机，将于2023年发布，售价89.99美元。它有一个400W的电机，500ml的容量，由不含bpa的塑料制成。它有2年的保修期，有三种颜色可供选择：蓝色、白色和绿色。该产品专为户外使用而设计，可以在10秒内混合冰。"
prompt=f"""
## 文本内容
{text}

## 问题
请你从上述文本中总结EcoFusion Pro需要多少钱？
"""

In [4]:
print(inference(model_instruct,tokenizer_instruct,prompt))

{'response': 'EcoFusion Pro的价格是89.99美元。', 'generated_tokens_count': 15}
